# Práctica Calificada 2
## Predicción de tarifas de taxis
El objetivo de esta evaluación es construir un modelo de aprendizaje que sea capaz de predecir la tarifa que cobra un taxi de acuerdo a cierta información de entrada.


In [ ]:
import pandas as pd
import numpy as np
print("Pandas = ", pd.__version__)
print("Numpy = ", np.__version__)

# Obteniendo del conjunto de datos

In [ ]:
pd.read_csv("./train.csv",nrows=5).head()

Tenemos las siguientes columnas

*   key: cadena que identifica de manera única a cada registro.
*   fare_amount: número real indicando el costo del taxi. Esta es la variable a predecir.
*   pickup_datetime: timestamp indicando cuando el viaje a empezado.
*   pickup_longitude: número real indicando la ubicación en longitud en donde el viaje empezó.
*   pickup_latitude: número real indicando la ubicación en latitud en donde el viaje empezó.
*   dropoff_longitude: número real indicando la ubicación en longitud en donde el viaje terminó.
*   dropoff_latitude: número real indicando la ubicación en latitud en donde el viaje terminó.
*   passenger_count: número entero indicando el número de pasajeros en el servicio de taxi.

*La columna key no es un caracterísitica que nos interese.*

**Cargamos solo las columnas que nos interesan**

In [7]:
columns = ["fare_amount","pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count"]

In [8]:
df_train = pd.read_csv("./train.csv",nrows=50000000,usecols=columns)

**Uso total de la memoria al inicio**

In [ ]:
memoria_inicial = df_train.memory_usage(deep=True).sum()
print(f"{memoria_inicial/1e9:0.2f}GB")

**Analizando el tipo de las variables**

In [ ]:
df_train.dtypes

Uso de memoria de cada columna en bytes.

In [ ]:
df_train.memory_usage(deep=True)

**Utilizando tipos de datos eficientes.**

In [13]:
df_train["passenger_count"] = pd.to_numeric(df_train["passenger_count"], downcast="unsigned")
df_train["fare_amount"] =  pd.to_numeric(df_train["fare_amount"], downcast="float")
df_train[["pickup_longitude","pickup_latitude"]] = df_train[["pickup_longitude","pickup_latitude"]].apply(pd.to_numeric, downcast="float")
df_train[["dropoff_longitude","dropoff_latitude"]] = df_train[["dropoff_longitude","dropoff_latitude"]].apply(pd.to_numeric, downcast="float")

In [ ]:
df_train.dtypes

Uso de memoria de cada columna en bytes luego de utilizar tipos de datos más eficientes.

In [ ]:
df_train.memory_usage(deep=True)

Uso de memoria total luego de utilizar tipos de datos más eficientes.

In [ ]:
memoria_final = df_train.memory_usage(deep=True).sum()
print(f"{memoria_final/1e9:0.2f}GB")

**Reducción de Memoria**

In [ ]:
reduccion = (memoria_inicial-memoria_final)/memoria_inicial*100
print(f"Reducimos el uso de memoria en {reduccion:0.2f}%")

In [ ]:
df_train.describe()

In [ ]:
df_train.info(verbose=True, null_counts=True)

De lo anterior se puede observar que tenemos valores nulos, entonces verificamos cuando valores nulos hay por cada columna:

In [ ]:
print(df_train.isnull().sum())

In [22]:
# Eliminando
df_train = df_train.dropna(how='any', axis=0)

# Analizando Fare_amount

In [ ]:
df_train[['fare_amount']].describe()

Obervamos que el el minimo del monto de tarifa es negativo, veamos cuantos montos de tarifa menores o iguales a cero, tenemos: 

In [ ]:
len(df_train[df_train['fare_amount'] <= 0].index)

In [ ]:
df_train[df_train['fare_amount'] <= 0]

Solo queremos los datos donde el monto de la tarifa sea mayor que cero:


In [26]:
df_train = df_train[df_train['fare_amount'] > 0]

In [ ]:
df_train[['fare_amount']].describe()

# Analizando la longitud y latidud

El rango de la latidud es de -90 hasta 90 grados, mientras que el grado de la longitud es de -180 hasta los 180 grados.



In [28]:
df_train = df_train[(df_train['pickup_longitude'] >= -180) & (df_train['pickup_longitude'] <= 180)]

In [29]:
df_train = df_train[(df_train['pickup_latitude'] >= -90) & (df_train['pickup_latitude'] <= 90)]

In [30]:
df_train = df_train[(df_train['dropoff_longitude'] >= -180) & (df_train['dropoff_longitude'] <= 180)]

In [31]:
df_train = df_train[(df_train['dropoff_latitude'] >= -90) & (df_train['dropoff_latitude'] <= 90)]

In [32]:
def distancia(df):
  # Radio medio de la tierra en Km
  R = 6371.0
  # Conversion a radianes
  lt1 = np.radians(df.pickup_latitude)
  lg1 = np.radians(df.pickup_longitude)
  lt2 = np.radians(df.dropoff_latitude)
  lg2 = np.radians(df.dropoff_longitude)
  # Defirencia entre latitudes y longitudes
  dlt = lt2 - lt1
  dlg = lg2 - lg1
  # Haversine
  hav = np.sin(dlt / 2)**2 + np.cos(lt1) * np.cos(lt2) * np.sin(dlg / 2)**2
  c = 2 * np.arctan2(np.sqrt(hav), np.sqrt(1-hav))
  d = R * c
  return d

In [33]:
df_train['distancia'] = distancia(df_train)

In [34]:
df_train = df_train.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

Limpiando data para distancia igual a cero kilometros o demasiado extensas

In [ ]:
df_train[['distancia']].describe()

In [ ]:
df_train['distancia'] = df_train[(0 < df_train['distancia']) & (df_train['distancia'] < 70)]
df_train[['distancia']].describe()

# Analizando pickup_datetime

In [ ]:
df_train['pickup_datetime']

In [ ]:
df_train['pickup_datetime'] = df_train['pickup_datetime'].str.replace(" UTC", "")

In [ ]:
df_train['pickup_datetime'] = pd.to_datetime(df_train['pickup_datetime'])

In [ ]:
df_train['año'] = df_train.pickup_datetime.dt.year
df_train['mes'] = df_train.pickup_datetime.dt.month
df_train['dia'] = df_train.pickup_datetime.dt.day
df_train['hora'] = df_train.pickup_datetime.dt.hour

In [ ]:
df_train = df_train.drop(columns=['pickup_datetime'])

In [ ]:
df_train.head()

# Analizando Cantidad de pasajeros

In [ ]:
df_train[['passenger_count']].describe()

Eliminamos la data que contenga cantidad de pasajeros mayor o igual a 6

In [ ]:
df_train['passenger_count'] = df_train[(0 < df_train['passenger_count']) & (df_train['passenger_count'] < 6)]
df_train[['passenger_count']].describe()

# Entrenamieto

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

predictors = ['passenger_count', 'distancia', 'año', 'mes', 'dia', 'hora']
salida = 'fare_amount'

X = df_train[predictors]
y = df_train[salida]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=22)

rf = RandomForestRegressor(n_estimators=500,
                            oob_score = True,
                            random_state=1,
                            max_depth=8)
rf.fit(X_train, y_train)

In [ ]:
from joblib import dump, load
dump(rf, 'Modelo.joblib')